<a href="https://colab.research.google.com/github/prinzessinmarlenifee/SenseCap/blob/main/SenseCap_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imu Cnn Training Pipeline für 3 Sensoren pro Session (head, wrist, seat)
# Leave-One-Session-Out Validierung (18 Sessions) mit CNN+LSTM bei 60 Hz Samplingrate

Hinweise zur Ausführung:

    Alle Abhängigkeiten installieren (Zelle 3) – anschließend brauchst du das nur einmal pro Notebook.

    Drive mounten (Zelle 2) immer als erstes ausführen, um Zugriff auf /content/drive/... zu haben.

    Pfade anpassen:

        Stelle sicher, dass base_dir = '/content/drive/MyDrive/IMU_Sessions/' korrekt ist.

        In jedem Unterordner (SessionXX) müssen genau vier Dateien liegen:

            hot.json

            head.csv

            wrist.csv

            seat.csv

    Code-Zellen einzeln ausführen (Zelle 5 kann komplett auf einmal ausgeführt werden).
    → Nach kurzer Zeit (abhängig von GPU) siehst du pro Session die Anzahl der Fenster und die Test-Accuracy.

5. Was passiert Schritt für Schritt im Code?

    session_dirs = sorted([...])
    Liest die 18 Unterordner ein.

    parse_hot_labels(...)
    Wandelt die Zeitstempel in hot.json in ein frame_labels-Array um (Länge = Anzahl der Zeilen in head.csv).

    window_data_multiple_sensors(...)
    Erzeugt für jeden 60-Frame-Abschnitt (1 s bei 60 Hz) je Sensor ein Array mit 6 Features, kombiniert drei Sensoren zu 18 Features und weist ein dominantes Label zu (ignoriert „Unknown“).

    Daten sammeln
    Für jede Session werden X_windows (Fenster) und y_windows (Labels) in Listen sessions_X und sessions_y gespeichert.

    Leave-One-Session-Out-Schleife

        Eine Session wird als Testset unbeachtet behalten.

        Alle anderen Sessions werden zu X_train, y_train zusammengefügt.

        Auf y_train wird ein LabelEncoder() gefittet; dieselbe Kodierung wird auf y_test angewandt.

    Modell-Definition

        Zwei Conv1D-Schichten (jeweils 64 Filter, Kernelgröße 3)

        MaxPooling1D(pool_size=2) → halbiert die Zeitschritt-Dimension

        Dropout(0.3) → verhindert Overfitting

        LSTM(64) → lernt zeitliche Abhängigkeiten

        Dense(100) + Dense(n_classes) → endgültige Klassifikation

    Training

        epochs=20, batch_size=32, validation_split=0.1

        Modell sieht 20-mal alle Trainingsfenster (aufgeteilt in Batches à 32)

    Evaluation

        model.evaluate(X_test, y_test_enc) liefert Test-Accuracy.

        classification_report(...) zeigt Precision/Recall/F1 pro Klasse.

        ConfusionMatrixDisplay plottet die Matrix für die jeweilige Test-Session.

    Zusammenfassung

        Durchschnittliche Genauigkeit über alle 18 Sessions

        Export als session_accuracy_summary.csv

In [2]:
# --- PARAMETER ---
sampling_rate = 60       # 60 Hz nach SDI-Algorithmus
window_size = 60         # 1 Sekunde = 60 Frames
step_size = 30           # 50% Überlappung

# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive')

# Basisverzeichnis, in dem alle Session-Ordner liegen
base_dir = '/content/drive/MyDrive/ML-MTB-Modell/IMU-Sessions/'  #Drive-Pfad


import os

# Prüfe, ob das Verzeichnis existiert und zeige die ersten Sessions an
if os.path.exists(base_dir):
    print("Verzeichnis gefunden. Enthält folgende Ordner/Sessions:")
    print(sorted(os.listdir(base_dir))[:10], "…")
else:
    print("Achtung: Pfad existiert nicht, bitte base_dir anpassen!")



Mounted at /content/drive
Verzeichnis gefunden. Enthält folgende Ordner/Sessions:
['Session_01', 'Session_02', 'Session_03', 'Session_04', 'Session_05', 'Session_06', 'Session_07', 'Session_08', 'Session_09', 'Session_10'] …


In [ ]:

#google-Drive inhalt anzeigen für Überblick/Überbrüfen
!ls /content/drive/MyDrive/

 championship_data.npy	   input_video.mp4     ptbdb_abnormal.csv
'Colab Notebooks'	   Jump_Data	       ptbdb_normal.csv
 Einstelldatenblatt.gdoc   ML-MTB-Modell      'Unbenanntes Formular.gform'
 Einstelldatenblatt.pdf    picture_ski_2.jpg
 hurensohnbild.jpg	   picture_ski.jpg


In [3]:
import os
import json
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import tensorflow as tf
layers = tf.keras.layers
models = tf.keras.models


# --- PARAMETER ---
sampling_rate = 60       # 60 Hz nach SDI-Algorithmus
window_size = 60         # 1 Sekunde = 60 Frames
step_size = 30           # 50% Überlappung

# Basisverzeichnis (sollte die 18 Session-Ordner enthalten)
base_dir = '/content/drive/MyDrive/ML-MTB-Modell/IMU-Sessions/'


# Session-Ordner erkennen
session_dirs = sorted([
    d for d in os.listdir(base_dir)
    if os.path.isdir(os.path.join(base_dir, d))
])
print(f"Gefundene Sessions: {len(session_dirs)} -> {session_dirs}")

# Funktion: Hot-Labels pro Frame generieren
def parse_hot_labels(json_path, total_frames):
    with open(json_path, 'r') as f:
        data = json.load(f)
    button_press_str = data['button_presses']
    entries = button_press_str.strip().split(';')

    label_changes = []
    for entry in entries:
        if ':' in entry:
            label, frame = entry.strip().split(':')
            label = label.strip()
            # Falsche Schreibweise korrigieren
            if label.lower() == 'peadling':
                label = 'Pedaling'
            label_changes.append((int(frame.strip()), label))

    # Standardmäßig 'Unknown' setzen
    frame_labels = ['Unknown'] * total_frames
    for i, (start_frame, label) in enumerate(label_changes):
        end_frame = label_changes[i + 1][0] if i + 1 < len(label_changes) else total_frames
        for f in range(start_frame, min(end_frame, total_frames)):
            frame_labels[f] = label
    return frame_labels

# Funktion: Fensterung & Label-Zuweisung für 3 Sensoren zusammengeführt
def window_data_multiple_sensors(head_data, wrist_data, seat_data, frame_labels):
    X_windows, y_windows = [], []
    total_frames = len(frame_labels)

    # Annahme: head_data, wrist_data, seat_data haben alle dieselbe Anzahl Zeilen
    for start in range(0, total_frames - window_size + 1, step_size):
        end = start + window_size
        # Fenster je Sensor: (window_size, 6)
        win_h = head_data[start:end]
        win_w = wrist_data[start:end]
        win_s = seat_data[start:end]
        # Kombiniere: (window_size, 18) – 3 Sensoren • 6 Achsen
        window = np.concatenate([win_h, win_w, win_s], axis=1)

        # Label-Fenster
        label_window = frame_labels[start:end]
        label_counts = Counter(label_window)
        dominant_label = label_counts.most_common(1)[0][0]
        if dominant_label == 'Unknown':
            continue

        X_windows.append(window)
        y_windows.append(dominant_label)

    return np.array(X_windows), np.array(y_windows)

# --- DATEN SAMMELN: alle Sessions laden, Fenster und Labels erzeugen ---
sessions_X = []
sessions_y = []

for sess_dir in session_dirs:
    print(f"\nLade Session: {sess_dir}")
    session_path = os.path.join(base_dir, sess_dir)

    # 1) Sensor-Dateien anhand ihres Prefix finden
    def find_sensor_file(folder, prefix):
        for f in os.listdir(folder):
            if f.lower().startswith(prefix.lower()):
                return os.path.join(folder, f)
        raise FileNotFoundError(f"❌ Keine Datei mit Prefix '{prefix}' in {folder} gefunden.")

    head_path  = find_sensor_file(session_path, 'Head_')
    wrist_path = find_sensor_file(session_path, 'Wrist_')
    seat_path  = find_sensor_file(session_path, 'Seat_')

    # 2) IMU-Daten laden: Kopf, Handgelenk, Sitz (je 6 Spalten)
    head_data  = pd.read_csv(head_path).values   # numpy array, shape (n_frames, 6)
    wrist_data = pd.read_csv(wrist_path).values
    seat_data  = pd.read_csv(seat_path).values

    total_frames   = head_data.shape[0]


    # Funktion: finde Hot-Datei mit Suffix "_hot.json"
    def find_hot_file(folder):
        for f in os.listdir(folder):
            if f.lower().endswith('_hot.json'):
                return os.path.join(folder, f)
        raise FileNotFoundError(f"❌ Keine Datei mit Suffix '_hot.json' in {folder} gefunden.")
    # Labels aus hot.json erstellen
    hot_path = find_hot_file(session_path)

    frame_labels   = parse_hot_labels(hot_path, total_frames)

    # Beispiel: Synch-Datei einlesen (wenn vorhanden)
    synch_path = os.path.join(session_path, 'synch_data.json')
    if os.path.exists(synch_path):
        with open(synch_path, 'r') as f:
            synch = json.load(f)
        offset_head  = int(synch.get("Head Sensor Video (Timestamp) in Frames", 0))
        offset_wrist = int(synch.get("Wrist Sensor Video (Timestamp) in Frames", 0))
        offset_seat  = int(synch.get("Seat Sensor Video (Timestamp) in Frames", 0))

        head_data  = head_data[offset_head:]
        wrist_data = wrist_data[offset_wrist:]
        seat_data  = seat_data[offset_seat:]
        print(f"⏱ Synch angewendet: Head={offset_head}, Wrist={offset_wrist}, Seat={offset_seat}")
    else:
        print(f"ℹ️ Keine synch_data.json gefunden für {sess_dir}")
    # Kürzen auf gemeinsame Länge
    min_length = min(len(head_data), len(wrist_data), len(seat_data))
    head_data  = head_data[:min_length]
    wrist_data = wrist_data[:min_length]
    seat_data  = seat_data[:min_length]
    frame_labels = frame_labels[:min_length]

    # Fensterung & Labeling
    X_win, y_win = window_data_multiple_sensors(head_data, wrist_data, seat_data, frame_labels)

    sessions_X.append(X_win)
    sessions_y.append(y_win)
    print(f" → {len(X_win)} Fenster, {len(np.unique(y_win))} Klassen in Session {sess_dir}\n")





      # --- Session-Säuberung vor dem Training ---
    sessions_X_cleaned = []
    sessions_y_cleaned = []

    for x, y in zip(sessions_X, sessions_y):
        if x.shape[0] > 0 and len(x.shape) == 3:
            sessions_X_cleaned.append(x)
            sessions_y_cleaned.append(y)
        else:
            print(f"Session mit leerem oder ungültigem Shape übersprungen: {x.shape}")

    # Jetzt mit den bereinigten Sessions weiterarbeiten:
    sessions_X = sessions_X_cleaned
    sessions_y = sessions_y_cleaned

    print(f"Anzahl gültiger Sessions nach Filterung: {len(sessions_X)}")


    # Train-Daten: alle anderen Sessions
    X_train = np.concatenate([x for i, x in enumerate(sessions_X) if i != test_idx])
    y_train = np.concatenate([y for i, y in enumerate(sessions_y) if i != test_idx])

    # Label Encoding (gemeinsam auf Trainingsdaten fitten)
    le = LabelEncoder()
    y_train_enc = le.fit_transform(y_train)
    y_test_enc = le.transform(y_test)

    # Modell-Architektur (CNN + LSTM)
    model = models.Sequential([
        layers.Conv1D(64, 3, activation='relu', input_shape=X_train.shape[1:]),
        layers.Conv1D(64, 3, activation='relu'),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.3),
        layers.LSTM(64),
        layers.Dropout(0.3),
        layers.Dense(100, activation='relu'),
        layers.Dense(len(le.classes_), activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Training (mit 10% Validierungssplit aus Trainingsdaten)
    history = model.fit(
        X_train, y_train_enc,
        validation_split=0.1,
        epochs=20,
        batch_size=32,
        verbose=0
    )

    # Evaluation auf Test-Session
    test_loss, test_acc = model.evaluate(X_test, y_test_enc, verbose=0)
    print(f"✅ Test-Accuracy für {sess_name}: {test_acc:.2f}")
    all_accuracies.append(test_acc)
    accuracy_summary['Session'].append(sess_name)
    accuracy_summary['Accuracy'].append(test_acc)

    # Klassifikationsbericht
    y_pred_probs = model.predict(X_test, verbose=0)
    y_pred_classes = np.argmax(y_pred_probs, axis=1)

    print("\nKlassifikationsbericht:")
    print(classification_report(y_test_enc, y_pred_classes, target_names=le.classes_))

    # Confusion Matrix
    cm = confusion_matrix(y_test_enc, y_pred_classes)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
    disp.plot(xticks_rotation=45)
    plt.title(f"Confusion Matrix – {sess_name}")
    plt.show()

# --- Zusammenfassung über alle Sessions ---
mean_acc = np.mean(all_accuracies)
print("\n📈 Gesamtergebnis: Leave-One-Session-Out")
for sess, acc in zip(accuracy_summary['Session'], accuracy_summary['Accuracy']):
    print(f"  {sess}: {acc:.2f}")
print(f"\n✅ Durchschnittliche Genauigkeit über alle Sessions: {mean_acc:.2f}\n")

# Optional: Speichere Ergebnis als CSV
df_summary = pd.DataFrame(accuracy_summary)
output_csv = 'session_accuracy_summary.csv'
df_summary.to_csv(output_csv, index=False)
print(f"📁 Zusammenfassung gespeichert in: {output_csv}")


Gefundene Sessions: 14 -> ['Session_01', 'Session_02', 'Session_03', 'Session_04', 'Session_05', 'Session_06', 'Session_07', 'Session_08', 'Session_09', 'Session_10', 'Session_11', 'Session_12', 'Session_13', 'Session_14']

Lade Session: Session_01
⏱ Synch angewendet: Head=1386, Wrist=1236, Seat=1035
 → 1951 Fenster, 3 Klassen in Session Session_01

Anzahl gültiger Sessions nach Filterung: 1


NameError: name 'test_idx' is not defined

In [ ]:
#print shape for debug


  #ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 6 has 1 dimension(s)
for i, x in enumerate(sessions_X):
    print(f"Session {i}: shape = {np.array(x).shape}")

Session 0: shape = (1951, 60, 6)
Session 1: shape = (1546, 60, 6)
Session 2: shape = (1379, 60, 6)
Session 3: shape = (1836, 60, 6)
Session 4: shape = (1822, 60, 6)
Session 5: shape = (2079, 60, 6)
Session 6: shape = (1965, 60, 6)
Session 7: shape = (0,)
Session 8: shape = (1948, 60, 6)
Session 9: shape = (2037, 60, 6)
Session 10: shape = (2157, 60, 6)
Session 11: shape = (1946, 60, 35)
Session 12: shape = (2144, 60, 35)
Session 13: shape = (1810, 60, 35)


In [ ]:
from collections import Counter
shapes = [x.shape for x in sessions_X]
feature_counts = [shape[2] if len(shape) == 3 else None for shape in shapes]
print(Counter(feature_counts))

Counter({6: 1})


In [ ]:
if len(x_synced) > 0:
    sessions_X.append(x_synced)

NameError: name 'x_synced' is not defined

5. Was passiert Schritt für Schritt im Code?

    session_dirs = sorted([...])
    Liest die 18 Unterordner ein.

    parse_hot_labels(...)
    Wandelt die Zeitstempel in hot.json in ein frame_labels-Array um (Länge = Anzahl der Zeilen in head.csv).

    window_data_multiple_sensors(...)
    Erzeugt für jeden 60-Frame-Abschnitt (1 s bei 60 Hz) je Sensor ein Array mit 6 Features, kombiniert drei Sensoren zu 18 Features und weist ein dominantes Label zu (ignoriert „Unknown“).

    Daten sammeln
    Für jede Session werden X_windows (Fenster) und y_windows (Labels) in Listen sessions_X und sessions_y gespeichert.

    Leave-One-Session-Out-Schleife

        Eine Session wird als Testset unbeachtet behalten.

        Alle anderen Sessions werden zu X_train, y_train zusammengefügt.

        Auf y_train wird ein LabelEncoder() gefittet; dieselbe Kodierung wird auf y_test angewandt.

    Modell-Definition

        Zwei Conv1D-Schichten (jeweils 64 Filter, Kernelgröße 3)

        MaxPooling1D(pool_size=2) → halbiert die Zeitschritt-Dimension

        Dropout(0.3) → verhindert Overfitting

        LSTM(64) → lernt zeitliche Abhängigkeiten

        Dense(100) + Dense(n_classes) → endgültige Klassifikation

    Training

        epochs=20, batch_size=32, validation_split=0.1

        Modell sieht 20-mal alle Trainingsfenster (aufgeteilt in Batches à 32)

    Evaluation

        model.evaluate(X_test, y_test_enc) liefert Test-Accuracy.

        classification_report(...) zeigt Precision/Recall/F1 pro Klasse.

        ConfusionMatrixDisplay plottet die Matrix für die jeweilige Test-Session.

    Zusammenfassung

        Durchschnittliche Genauigkeit über alle 18 Sessions

        Export als session_accuracy_summary.csv